In [ ]:
import importlib

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# BERT Section Below

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
import re
from langdetect import detect

# df = pd.read_csv('train.csv')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/full_dataset_50pcGen.csv')

# Create a new DataFrame to store the filtered lines
filtered_df = pd.DataFrame(columns=df.columns)

# Iterate through the original DataFrame
for i in range(len(df)):
    try:
        line = str(df.iloc[i]['title'])  # Replace 'title' with the actual column name containing the title
        language = detect(line)

        if language == 'en':
            # Append the line to the filtered DataFrame if the language is English
            filtered_df = filtered_df.append(df.iloc[i], ignore_index=True)
            #print(f"Line {i+1}: Language - {language} (Included)")
        else:
            #print(f"Line {i+1}: Language - {language} (Excluded)")
            pass
    except TypeError:
        print(f"Line {i+1}: Skipped - Non-string text")
        continue

# Save the filtered DataFrame to a new file
filtered_df.to_csv('news_filtered.csv', index=False)
df = pd.read_csv('news_filtered.csv')





<ipython-input-5-aa3998d03e17>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.append(df.iloc[i], ignore_index=True)


In [ ]:
# Step 2: Preprocessing
# Here, we are not performing text cleaning steps as BERT benefits from the full context of the sentence
df = df.dropna(subset=['text'])
df = df.dropna(subset=['title'])


# Step 3: BERT Tokenization & Formatting
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_text(df, max_len):
    return tokenizer.batch_encode_plus(
        df['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

In [ ]:
# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)

# Step 4: Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# Split into training and testing datasets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)


In [ ]:
# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)



In [ ]:
# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)



In [ ]:
# Step 5: Training
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Step 6: Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Move logits and labels to CPU
    logits = outputs[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# For each input batch, pick the label (0 or 1) with the higher score
pred_flat = np.argmax(predictions, axis=1).flatten()

# Print the classification report
print(classification_report(true_labels, pred_flat, target_names=['Fake News', 'Real News']))

              precision    recall  f1-score   support

   Fake News       0.00      0.00      0.00      2054
   Real News       0.46      1.00      0.63      1785

    accuracy                           0.46      3839
   macro avg       0.23      0.50      0.32      3839
weighted avg       0.22      0.46      0.30      3839



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
